Скачивание всех зависимостей проекта

In [4]:
!pip install -r requirements.txt

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import random
import requests
import json
from bs4 import BeautifulSoup

In [2]:
# конфигурация
with open('config.json', 'r') as file:
    config = json.load(file)

data_path = config['data_path']
data_path_without_repeats = config['data_path_without_repeats']
data_url = config['data_url']

In [3]:
paronyms = []

# взятие паронимов с открытого ресурса
ru_alphabet = [chr(i) for i in range(ord('А'), ord('А') + 32)]
no_words = ['Ё', 'Й', 'Щ', 'Ъ', 'Ы', 'Ь']

# Создаем множество всех слов из датасета паронимов
all_words = set()

for letter in ru_alphabet:
    if letter in no_words:
        continue
    url = data_url + '/' + letter
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    arr = soup.find_all('a', href=True)[2:-27]  # Откидываем первые 2 и последние 27 элемента, которые есть на каждой странице
    for elem in arr:
        webpage = data_url + elem['href']
        words = (elem.string).split(' — ')
        paronyms.append([words[0], words[1], 1])  # пары слов - паронимы
        all_words.update((words[0], words[1])) # Создаем множество всех слов из датасета паронимов

In [4]:
# база данных с кортежами с меткой (1) - паронимы
print(paronyms[:5])

[['абонемент', 'абонент', 1], ['абстрактность', 'абстракция', 1], ['авантюрин', 'авантюрист', 1], ['автобиографический', 'автобиографичный', 1], ['автоматический', 'автоматичный', 1]]


In [5]:
data = pd.DataFrame(paronyms, columns=['word1', 'word2', 'label'])
data.to_csv(data_path_without_repeats)

In [10]:
# создание пар слов не паронимов путем перемешивания уже имеющихся слов
def generate_non_paronyms(paronyms, seed = None, ratio = 1):

    non_paronyms = []

    if seed:
      random.seed(seed)

    # Случайные пары из множества всех слов
    for _ in range(int(len(paronyms) * ratio )):
        word1, word2 = random.choices(list(all_words), k=2)
        if (word1 + ' ' + word2, 1) not in paronyms:
          non_paronyms.append([word1, word2, 0])  # Метка 0, так как это непаронимы

    return non_paronyms

non_paronyms = generate_non_paronyms(paronyms, 42)

In [11]:
dataset_par = paronyms + non_paronyms
random.shuffle(dataset_par) # перемешивание датасета на всякий случай
print(dataset_par[:5])

[['обездушить', 'комичный', 0], ['целый', 'цельный', 1], ['сравнимый', 'неорганичный', 0], ['чубастый', 'завоевательный', 0], ['сыскать', 'снискать', 1]]


In [12]:
data = pd.DataFrame(dataset_par, columns=['word1', 'word2', 'label'])
data.to_csv(data_path)